In [ ]:
import numpy as np
import torch
from torch import nn
import matplotlib
import matplotlib.pyplot as plt
import tempfile
#import Experiments.Foong_L5W50.setup as exp
import argparse
import math

import seaborn as sns
import pandas as  pd

In [ ]:
# Find what device to work with
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

## Generate Boston Housing Dataset .pt (only run once)

In [ ]:
data = np.loadtxt('Experiments/Boston/Data/boston_housing.txt')
X = data[:, range(data.shape[ 1 ] - 1) ]
y = data[:, data.shape[ 1 ] - 1 ]

permutation = np.random.choice(range(X.shape[0]), X.shape[0], replace = False)
size_train = np.round(X.shape[0]*0.9)
index_train = permutation[0:int(size_train)]
index_test = permutation[int(size_train) : ]

X_train = X[ index_train, : ]
y_train = y[ index_train ]
X_test = X[ index_test, : ]
y_test = y[ index_test ]

X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

torch.save(X_train, 'Experiments/Boston/Data/boston_train_X.pt')
torch.save(y_train, 'Experiments/Boston/Data/boston_train_y.pt')

torch.save(X_test, 'Experiments/Boston/Data/boston_test_X.pt')
torch.save(y_test, 'Experiments/Boston/Data/boston_test_y.pt')

In [ ]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston

# Data #

In [ ]:
x_train = torch.load('Experiments/Boston/Data/boston_train_X.pt').to(device).float()
y_train = torch.load('Experiments/Boston/Data/boston_train_y.pt').to(device).float()
x_test = torch.load('Experiments/Boston/Data/boston_test_X.pt').to(device).float()
y_test = torch.load('Experiments/Boston/Data/boston_test_y.pt').to(device).float()
y_train = y_train.unsqueeze(-1)

# Predictive Net #

In [ ]:
input_dim=13
nblayers = 1
activation=nn.Tanh()
layerwidth = 50
param_count = (input_dim+1)*layerwidth+(nblayers-1)*(layerwidth**2+layerwidth)+layerwidth+1

In [ ]:
def mlp(x,theta,input_dim=input_dim,layerwidth=layerwidth,nb_layers=nblayers,activation=activation):
    """
    Feedforward neural network used as the observation model for the likelihood
    

    Parameters:
        x (Tensor): Input of the network of size NbExemples X NbDimensions   
        theta (Tensor):  M set of parameters of the network NbModels X NbParam
        input_dim (Int): dimensions of NN's inputs (=NbDimensions)
        layerwidth (Int): Number of hidden units per layer 
        nb_layers (Int): Number of layers
        activation (Module/Function): activation function of the neural network

    Returns:
        Predictions (Tensor) with dimensions NbModels X NbExemples X NbDimensions


    Example:

    input_dim=11
    nblayers = 2
    activation=nn.Tanh()
    layerwidth = 20
    param_count = (input_dim+1)*layerwidth+(nblayers-1)*(layerwidth**2+layerwidth)+layerwidth+1

    x=torch.rand(3,input_dim)
    theta=torch.rand(5,param_count)
    mlp(x,theta,input_dim=input_dim,layerwidth=layerwidth,nb_layers=nblayers,activation=activation)

    """

    nb_theta=theta.shape[0]
    nb_x=x.shape[0]
    split_sizes=[input_dim*layerwidth]+[layerwidth]+[layerwidth**2,layerwidth]*(nb_layers-1)+[layerwidth,1]
    theta=theta.split(split_sizes,dim=1)
    input_x=x.view(nb_x,input_dim,1)
    m=torch.matmul(theta[0].view(nb_theta,1,layerwidth,input_dim),input_x)
    m=m.add(theta[1].reshape(nb_theta,1,layerwidth,1))
    m=activation(m)
    for i in range(nb_layers-1):
        m=torch.matmul(theta[2*i+2].view(-1,1,layerwidth,layerwidth),m)
        m=m.add(theta[2*i+3].reshape(-1,1,layerwidth,1))
        m=activation(m)
    m=torch.matmul(theta[2*(nb_layers-1)+2].view(nb_theta,1,1,layerwidth),m)
    m=m.add(theta[2*(nb_layers-1)+3].reshape(nb_theta,1,1,1))
    return m.squeeze(-1)

# Posterior density #

In [ ]:
sigma_noise = torch.tensor(0.1)

In [ ]:
def _log_norm(x, mu, std):
    """
    Evaluation of 1D normal distribution on tensors

    Parameters:
        x (Tensor): Data tensor
        mu (Tensor): Mean tensor of same size as x
        std (Tensor): Positive scalar

    Returns:
        logproba (Tensor): Same size as x
    """
    return -0.5 * torch.log(2*np.pi*std**2) -(0.5 * (1/(std**2))* (x-mu)**2)

In [ ]:
S = torch.ones(param_count).to(device)
mu = torch.zeros(param_count).to(device)
dim=param_count
def logprior(x,m=mu, s=S,dim=dim):
    n_x=x.shape[0]
    H=S.view(dim,1,1).inverse().view(1,1,dim)
    d=((x-mu.view(1,dim))**2).view(1,n_x,dim)
    const=0.5*S.log().sum()+0.5*dim*torch.tensor(2*math.pi).log()
    return -0.5*(H*d).sum(2)-const
#test!
#prior=torch.distributions.multivariate_normal.MultivariateNormal(mu,covariance_matrix=torch.diagflat(S))
#x=torch.rand(10,param_count)
#torch.allclose(logprior(x),prior.log_prob(x))
#True!

In [ ]:
def loglikelihood(theta, x, y, sigma_noise):
        y_pred = mlp(x, theta)
        L = _log_norm(y_pred, y.unsqueeze(0).repeat(theta.shape[0], 1, 1), torch.tensor([sigma_noise], device=device))
        return torch.sum(L, dim=1)

In [ ]:
def logposterior(theta, x, y, sigma_noise):
        return logprior(theta).add(loglikelihood(theta, x, y, sigma_noise))

In [ ]:
logtarget = lambda theta : logposterior(theta, x_train, y_train, sigma_noise )

# Hypernet #

In [ ]:
ensemble_size=1
lat_dim=5
init_w=.15
init_b=.001
activation=nn.ReLU()
output_dim=param_count
#x_train, y_train = exp.get_training_data(device)

In [ ]:
#x_train = x_train[:,:1].float()

In [ ]:
#y_train = y_train.float()

In [ ]:
class HNet(nn.Module):
            def __init__(self, lat_dim, nb_neur, output_dim,  activation=nn.ReLU(), init_w=.1, init_b=.1):
                super(HNet, self).__init__()
                self.lat_dim = lat_dim
                self.output_dim=output_dim
                self.hnet=nn.Sequential(
                        nn.Linear(lat_dim,nb_neur),
                        activation,
                        nn.Linear(nb_neur,output_dim)
                        ).to(device)
                
                torch.nn.init.normal_(self.hnet[2].weight,mean=0., std=init_w)
                torch.nn.init.normal_(self.hnet[2].bias,mean=0., std=init_b)
    
            def forward(self, n=1):
                epsilon = torch.randn(size=(n,self.lat_dim)).to(device)
                return self.hnet(epsilon)           

class HyNetEns(nn.Module):
    def __init__(self,nb_comp,lat_dim, output_dim, activation, init_w,init_b):
        super(HyNetEns, self).__init__()
        self.nb_comp=nb_comp
        self.output_dim=output_dim
        self.components= nn.ModuleList([HNet(lat_dim,output_dim,output_dim,activation,init_w,init_b) for i in range(nb_comp)]).to(device)   

        # "Silverman's rule of thumb", Wand and Jones p.111 "Kernel Smoothing" 1995.                                 
    def get_H(self, nb_samples):
        theta=self.sample(nb_samples)
        c_=(nb_samples*(self.output_dim+2))/4
        c=torch.as_tensor(c_).pow(2/(self.output_dim+4)).to(device)      
        H_=theta.var(1)/c
        #H_=(theta.var()/c)*torch.ones(self.output_dim) #to try!
        return theta, H_.clamp(torch.finfo().eps,float('inf'))

    def KDE(self, theta_,theta, H_):
        def kernel(theta1,theta2,H):
            mvn = torch.distributions.multivariate_normal.MultivariateNormal(theta1, torch.diagflat(H))
            return mvn.log_prob(theta2)
        LQ=torch.Tensor(theta_.shape[0],self.nb_comp,theta.shape[1]).to(device) 
        for c in range(self.nb_comp):
            for i in range(theta_.shape[0]):
                LQ[i,c]=kernel(theta_[i],theta[c],H_[c])
        N_=self.nb_comp*theta.shape[1]
        N=torch.as_tensor(float(N_)).to(device)
        return (LQ.logsumexp(2).logsumexp(1)-torch.log(N)).unsqueeze(1)

    def sample(self, n=1):
        return torch.stack([self.components[c](n) for c in range(self.nb_comp)])

    
    def forward(self, n=1):
        d = torch.distributions.multinomial.Multinomial(n, torch.ones(self.nb_comp))
        m = d.sample()
        return torch.cat([self.components[c](int(m[c])) for c in range(len(self.components))])

In [ ]:
Hyper_Nets=HyNetEns(ensemble_size,lat_dim, output_dim,activation,init_w,init_b).to(device)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

theta =Hyper_Nets.sample(500).detach().numpy()

fig, axs = plt.subplots(6, 3,figsize=(8,15))
for k in range(6):
    for l in range(3):
        i=np.random.choice(a=np.arange(param_count), size=2,replace=False)
        axs[k, l].set(title=str(i[0])+' vs '+str(i[1]))
        for c in range(Hyper_Nets.nb_comp):
            axs[k, l].scatter(theta[c,:,i[0]],theta[c,:,i[1]],marker='.',alpha=0.2,color='C'+str(c+2))

fig.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib notebook

x_lin = torch.arange(-2.,2.0,.01).unsqueeze(1).to(device)
# Sampling the distribution over Neural Networks 1000 times, and plotting with transparency to make it appear as a smooth distribution
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
#plt.title('Prior')
#ax.set_ylim(-4,4)

plt.scatter(x_train.cpu(), y_train.cpu())

nb_samples_plt=10
theta = Hyper_Nets.sample(nb_samples_plt).detach()
for c in range(Hyper_Nets.nb_comp):
    for i in range(nb_samples_plt):
        y_pred = mlp(x_lin,theta[c,i].unsqueeze(0))
    #    plt.plot(x_test.detach().cpu().numpy(), y_test.squeeze(0).detach().cpu().numpy(), alpha=0.05, linewidth=1, color='green')
        plt.plot(x_lin.detach().cpu().numpy(), y_pred.squeeze(0).detach().cpu().numpy(), alpha=0.05, linewidth=1, color='C'+str(c+2))
    

# Entropy estimation #

In [ ]:
def KDE(x,y,prec):
    """
    KDE    
    
    Parameters:
        x (Tensor): Inputs, NbExemples X NbDimensions   
        y (Tensor):  Batched samples, NbBatch x NbSamples X NbDimensions
        prec (Float): scalar factor for bandwidth scaling
    

    Returns:
        (Tensor) KDE estimate for x based on batched diagonal "Silverman's rule of thumb", NbExemples X 1
        See Wand and Jones p.111 "Kernel Smoothing" 1995.  
    
    """
    
    dim=x.shape[-1]
    n_ed=x.shape[0]
    n_comp=y.shape[0]
    n_kde=y.shape[1]
    c_=(n_kde*(dim+2))/4
    c=torch.as_tensor(c_).pow(2/(dim+4)).to(device)  
    H=prec*(y.var(1)/c).clamp(torch.finfo().eps,float('inf'))

    d=((y.view(n_comp,n_kde,1,dim)-x.view(1,1,n_ed,dim))**2)
    H_=H.view(n_comp,dim,1,1).inverse().view(n_comp,1,1,dim)
    const=0.5*H.log().sum(1)+0.5*dim*torch.tensor(2*math.pi).log()
    const=const.view(n_comp,1,1)
    ln=-0.5*(H_*d).sum(3)-const
    N=torch.as_tensor(float(n_comp*n_kde),device=device)
    return (ln.logsumexp(0).logsumexp(0)-torch.log(N)).unsqueeze(-1)


def NNE(theta,k=1):
    """
    Parameters:
        theta (Tensor): Samples, NbExemples X NbDimensions   
        k (Int): ordinal number

    Returns:
        (Float) k-Nearest Neighbour Estimation of the entropy of theta  
    
    """
    nb_samples=theta.shape[0]
    dim=theta.shape[1]
    D=torch.cdist(theta,theta)
    a = torch.topk(D, k=k+1, dim=0, largest=False, sorted=True)[0][k].clamp(torch.finfo().eps,float('inf')).to(device)
    d=torch.as_tensor(float(dim),device=device)
    K=torch.as_tensor(float(k),device=device)
    N=torch.as_tensor(float(nb_samples),device=device)
    pi=torch.as_tensor(math.pi,device=device)
    lcd = d/2.*pi.log() - torch.lgamma(1. + d/2.0)
    return torch.digamma(N) - torch.digamma(K) + lcd + d/nb_samples*torch.sum(torch.log(a))

In [ ]:
"""
Example of density estimation on Multivariate Normal

dim=100
p=torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(dim), torch.diagflat(2.*torch.ones(dim)))
print(p.entropy())
print(-KDE(p.sample((1,1000)).squeeze(0), p.sample((1,1000)),1.).mean())
print(NNE(p.sample((1,1000)).squeeze(0)))
"""

# Training #

In [ ]:
KDE_prec=1.
n_samples_KDE=500
n_samples_NNE=1000
n_samples_ED=10
n_samples_LP=10
max_iter=10000
learning_rate=0.001
min_lr=0.000005
patience=1000
lr_decay=0.9

In [ ]:
-KDE(Hyper_Nets(n_samples_ED),Hyper_Nets.sample(n_samples_KDE),KDE_prec).mean()

In [ ]:
NNE(Hyper_Nets(n_samples_NNE),1)

In [ ]:
from livelossplot import PlotLosses

%matplotlib inline

liveloss = PlotLosses()

In [ ]:
%matplotlib inline

optimizer = torch.optim.Adam(Hyper_Nets.parameters(), lr=learning_rate)
        
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=patience, factor=lr_decay)

        
for t in range(max_iter):
    optimizer.zero_grad()
    logs = {}


#KDE:    
    ED=-KDE(Hyper_Nets(n_samples_ED),Hyper_Nets.sample(n_samples_KDE),KDE_prec).mean()
#Nearest Neighbour Entropy estimation
#    ED=NNE(Hyper_Nets(n_samples_NNE))
    LP=logtarget(Hyper_Nets(n_samples_LP)).mean()
    L =-ED-LP

    L.backward()

    #training_loss.append(L.detach().clone().cpu().numpy())

    lr = optimizer.param_groups[0]['lr']

    logs['ELBO'] = L.detach().clone().cpu().numpy()
    logs['entropie diff'] = ED.detach().clone().cpu().numpy()
    logs['learning rate'] = learning_rate
    

    scheduler.step(L.detach().clone().cpu().numpy())

    optimizer.step()
    
    if t % 10 == 0:
        liveloss.update(logs)
        liveloss.draw()




    if lr < min_lr:
        break



# Metrics #

In [ ]:
#NLPD from Quinonero-Candela and al.
# the average negative log predictive density (NLPD) of the true targets

def NLPD(theta, x, y, sigma_noise):
    y_pred = mlp(x, theta)
    L = _log_norm(y_pred, y, torch.tensor([sigma_noise], device=device))
    n_x = torch.as_tensor(float(x.shape[0]), device=device)
    n_theta = torch.as_tensor(float(theta.shape[0]), device=device)
    log_posterior_predictive = torch.logsumexp(L, 0) - torch.log(n_theta)
    return torch.std_mean(log_posterior_predictive)

In [ ]:
lpp=NLPD(Hyper_Nets(1000),x_test,y_test,0.1)

In [ ]:
lpp

In [ ]:
X_train